<a href="https://colab.research.google.com/github/eliezerkapish/RAG-Architecture/blob/main/RAG_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Architecture

RAG Architecture is a framework combining information retrieval with natural language generation to enhance AI responses.

## Installing Packages

In [1]:
!pip install transformers sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 10.8 MB/s eta 0:00:00


## Importing Libraries

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

## Document database simulation

In [4]:
# Document database simulation

documents = [
    "Uranium is a vital component for the production of nuclear fuel, which is used in nuclear power reactors all over the world.",
    "Uranium is a naturally occurring radioactive element, which has the atomic number of 92 and corresponds to the chemical symbol U in the periodic table.",
    "Bitcoin is a form of digital currency that uses blockchain technology to support transactions between users on a decentralized network",
    ]

## Step 1 = Indexing documents using embeddings

In [5]:
# Step1 = Indexing documents using embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = embedder.encode(documents)

## FAISS criation

In [8]:
#FAISS criation
d= document_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(document_embeddings)

## Relevant document function

In [45]:
#Relevant document function
def retrieve_documents(query, k=2):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, k)
    retrieved_docs = [documents[idx] for idx in indices[0]]
    return retrieved_docs

## Loading Generator Model (Using T5 for summary)

In [46]:
#Loading Generator Model (Using T5 for summary)
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

## Answers Generator

In [47]:
# Answers Generator
def generate_answer(question, retrieved_docs):
  context = " ".join(retrieved_docs)
  input_text = f'Summarize: question: {question} context: {context}'
  inputs = tokenizer(input_text, return_tensors = 'pt', max_length=512, truncation=True)
  outputs = model.generate(**inputs, max_length=150, num_beams=5, early_stopping = True)
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return answer

## Use Example

In [48]:
# Use example
if __name__ == "__main__":
    question = "What is Bitcoin?"
    retrieved_docs = retrieve_documents(question)
    answer = generate_answer(question, retrieved_docs)
    print("Question:", question)
    print("Answer:", answer)

Question: What is Bitcoin?
Answer: digital currency
